# AdaBoost Regressor
## 1. Importación de librerías, establecimiento de directorio de trabajo y carga de datos

In [2]:
# librerias principales para uso y visualización de datos
import os
import numpy as np
import pandas as pd
import warnings  
warnings.filterwarnings('ignore')

# dividir datos para modelo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# algoritmo
from sklearn.ensemble import AdaBoostRegressor

# metricas de rendimiento
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold

# guardar el modelo
import pickle

# para visualización
import seaborn as sns  
import matplotlib.pyplot as plt  
from mpl_toolkits.mplot3d import Axes3D  
%matplotlib inline  

os.chdir("C:/Users/aleja/OneDrive - Universidad Politécnica de Madrid/Universidad/Asignaturas/Cuarto/Trabajo de Fin de Grado")

In [8]:
accesos = pd.read_csv("Resultados/Accesos_Calidad_Vida.csv", index_col="Unnamed: 0")
accesos.head()

,latitud,longitud,edad,calidad_vida
0,41.621468,2.068474,11,52.885748
1,41.600270,2.085002,12,74.793875
2,41.616524,2.089927,12,79.858657
3,41.618820,2.089480,11,79.858657
4,41.618908,2.089475,12,79.858657


## 2. Construcción de la matriz de características (X) y el vector variable dependiente (y)

In [9]:
X = accesos.iloc[:, :-1].values
y = accesos.iloc[:, -1].values

## 3. División del dataset en set de entrenamiento y de test

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

## 4. Feature scaling of X (rango: -3 a 3)

In [11]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## 5. Construcción y entrenamiento del modelo de regresión sobre el set de entrenamiento

In [3]:
regressor_lin = AdaBoostRegressor(n_estimators=700, loss='linear')
regressor_squ = AdaBoostRegressor(n_estimators=700, loss='square')
regressor_exp = AdaBoostRegressor(n_estimators=700, loss='exponential')

# entrenamos el modelo con los datasets de entrenamiento
regressor_lin.fit(X_train, y_train)
regressor_squ.fit(X_train, y_train)
regressor_exp.fit(X_train, y_train)

TypeError: __init__() got an unexpected keyword argument 'max_depth'

## 6. Predicción de resultados del set de test

In [23]:
y_pred1 = regressor_lin.predict(X_test)
y_pred2 = regressor_squ.predict(X_test)
y_pred3 = regressor_exp.predict(X_test)

## 7.  Métricas de rendimiento
Comparamos el set de test con el predicho

In [24]:
#Methods
print('\t\tLineal\t\tCuadrada\t\tExponencial')

# R2score
print('R2score:', 
      r2_score(y_test, y_pred1), 
      r2_score(y_test, y_pred2), 
      r2_score(y_test, y_pred3))

# Mean Absolute Error
print('MAE: \t', 
      mean_absolute_error(y_test, y_pred1), 
      mean_absolute_error(y_test, y_pred2), 
      mean_absolute_error(y_test, y_pred3))

# Mean Squared Error
print('MSE: \t', 
      mean_squared_error(y_test, y_pred1), 
      mean_squared_error(y_test, y_pred2), 
      mean_squared_error(y_test, y_pred3))

# Root Mean Squared Error
print("RMSE: \t", 
      mean_squared_error(y_test, y_pred1, squared=False), 
      mean_squared_error(y_test, y_pred2, squared=False), 
      mean_squared_error(y_test, y_pred3, squared=False))

		Lineal		Cuadrada		Exponencial
R2score: 0.9325344185558818 0.9257518651381038 0.6598465703523366
MAE: 	 4.5233292446112 4.793908154282619 10.967591191051532
MSE: 	 27.153260365971004 29.88307362714594 136.9035060826003
RMSE: 	 5.210879039660296 5.466541285597863 11.700577168781047


### 7.1. Para ganar más intuición

In [10]:
np.set_printoptions(precision=2) # redondea a dos decimales

print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

[[75.45 79.86]
 [75.45 79.86]
 [75.45 79.86]
 ...
 [75.45 79.86]
 [75.45 79.86]
 [75.45 79.86]]


### 7.2 Más métricas de rendimiento

In [12]:
accuracies = cross_val_score(estimator = regressor, X = X_train, y = y_train, cv = 10)

print("Media: {:.2f}".format(accuracies.mean()))
print("Desviacion Estandar: {:.2f}".format(accuracies.std()))

Media: 0.93 %
Desviacion Estandar: 0.01 %


Vemos que la precision y los errores no son muy buenos, por lo que probaremos otros parámetros para ver si ofrece alguno más óptimo
### 7.3 Grid Search

In [11]:
parameters = {
    'C': [1, 10, 100],
    'epsilon': [0.5, 1, 1.5]
}

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

grid_search = GridSearchCV(estimator = regressor,
                           param_grid = parameters,
                           scoring = 'r2',
                           cv = cv,
                           n_jobs = -1)

grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print("Best r2: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

KeyboardInterrupt: 

## 8. Guardar el mejor modelo

In [ ]:
with open('Modelos/AdaBoost.pkl','wb') as f:
    pickle.dump(grid_search,f)

## 9. Ejemplo de predicción

In [12]:
print(regressor.predict(sc.transform([['425060.955125', '4.606138e+06', '22']]))) 

[41.71]
